In [ ]:
from PIL import Image
import glob, os, fnmatch
from random import sample 
import toml

#debugging only
from IPython.core.debugger import set_trace

In [ ]:
# Allowbale files to be read
image_extensions = ['.jpeg', '.jpg', '.bmp', '.tif', '.png', '.gif']

# The program executable should be in the same folder as the 'folder_based_dojo'
content_directory = './folder_based_dojo/'

# Goes through the list of folders and extracts items for each category from their name
def get_inventory():

    # get all the files
    files = os.listdir(content_directory)

    # for each file, append it's name elements to a new list
    organ_list = []
    disease_type_list = []
    subtype_list = []
    complexity_list = []
    incidence_list = []

    for disease in os.listdir(content_directory):
        if disease.startswith('[') and disease.endswith(']'): # it's a disease
            category_list = disease.split('][')

            organ = category_list[0].strip('[')
            if organ not in organ_list:
                organ_list.append(organ)

            disease_type = category_list[1].strip('[')
            if disease_type not in disease_type_list:
                disease_type_list.append(disease_type)                        

            subtype = category_list[2].strip('[')
            if subtype not in subtype_list:
                subtype_list.append(subtype)

            complexity = category_list[3].strip('[')
            if complexity not in complexity_list:
                complexity_list.append(complexity)

            incidence = category_list[4].strip('[')
            if incidence not in incidence_list:
                incidence_list.append(incidence)            
            
    return organ_list, disease_type_list, subtype_list, complexity_list, incidence_list   

class Disease():

    def __init__(self, disease_folder_name):
        self.folder_name = disease_folder_name 
        self.images = self.get_list_of_images()   
        self.text_file_contents, self.name = self.load_text_file()   
        self.description = self.get_description()
        self.differentials = self.get_differentials()
        self.immunohistochemistry = self.get_immunohistochemistry()
        
    def load_text_file(self):
        contents = {} # start with blank dictionary to populate with toml data
        name = ''
        for file in os.listdir(content_directory+self.folder_name): 

            # if it is a text file
            if file.endswith('.txt') and not file.startswith('._'):                                        
                name = str(file).replace('.txt','')    
                try:
                    contents = toml.load(content_directory+self.folder_name+'/'+file) # load .txt as .toml
                except FileNotFoundError:
                    print('Error: could not load the .txt file')
                except:
                    print('There are formatting errors in the %s file')

        return contents, name


    def get_list_of_images(self):
        images = []
        for file in os.listdir(content_directory+self.folder_name): 
            if file.endswith(tuple(image_extensions)) and not file.startswith('._'):
                images.append(file)
        if images == 0:
            print("(no images found for "+self.folder_name)
        return images

            
    def get_description(self):
        description_text = []
        try:
            description_text = self.text_file_contents['description']
        except KeyError:
            print('(no description available)')        
            # for each file in the disease folder
        return description_text



    def show_all_images(self):
        try:
            for image in self.images:
                image_to_display = Image.open(content_directory+self.folder_name+'/'+image)
                image_to_display.show()    
        except:
            print('no images to display')

    def get_differentials(self):
        # This retrieves the differentials
        ddx_list = []
 
        try:
            ddx_list = self.text_file_contents['differentials']
        except KeyError:
            print('(no differentials available)')

        return ddx_list
    
    def display_differentials(self):
        # This prints the differentials out in a nice format
        print('There are %s differentials. They are:'%(len(self.differentials)))
        for individual_differential in self.differentials:
              print(individual_differential)
                     
                
    def get_immunohistochemistry(self):
        ihc_dictionary = {}
        try:
            ihc_dictionary = self.text_file_contents['immunohistochemistry']

        except KeyError:
            print('(no ihc available)')
        return ihc_dictionary

    def display_immunohistochemistry(self):        
        for ihc_name in self.immunohistochemistry:
            print('The %s is %s'%(ihc_name, self.immunohistochemistry[ihc_name]))
            
            
    def ask_for_answers(self):
        guess = str(input("*******************************\nYour answer is: "))

        if guess == '?':
            print('Here are some clues:\n\n')
            print(self.description)
            self.guess = str(input("*******************************\nYour answer is: "))
        elif guess == self.name:
            print('Correct! This is '+self.name)
        elif guess == 'q':
            return
        else:
            print('Correct answer: %s\n*******************************' %(self.name))
        print(self.description)        
        
        
    def differentials_challenge(self):
        # this starts a quiz based on the differential list of a disease
        differentials_quiz_length = len(self.differentials)
        
        print("Starting a differentials quiz for "+self.name)

        count = 1

        for individual_differential in self.differentials:
            print('Differential %s of %s.'%(count,len(self.differentials)))
            # find the folder name 
            if folder.endswith(str(individual_differential)+']'):
                try:
                   # turn into a disease
                    individual_differential = Disease(folder)
                    individual_differential.show_all_images()
                    individual_differentials.ask_for_answers()
                except: # folder doesn't exist
                    print('cannot find differential: '+folder)
            count+=1
        for ddx_name in self.get_differentials():
                print(ddx_name)        

        
def designed_quiz_preparer(organ = '', disease_type = '', subtype = '', complexity = '', incidence = '', 
                   name = '', number_to_quiz = None):
    
    # Get the parameters to search in the right order
    relevant_filenames = '[[]'+organ+'*'+disease_type+'*'+subtype+'*'+complexity+'*'+incidence+'[]]'
    
    
    temp = os.listdir(content_directory)
    shuffled_diseases = sample(temp, len(temp)) # shuffle the list to get a fresh quiz order
    filtered_diseases = fnmatch.filter(shuffled_diseases, relevant_filenames)

    if number_to_quiz == None:
        number_to_quiz = len(filtered_diseases)
    count = 0
    if number_to_quiz == 1:
        print('There is %s case to train: '%number_to_quiz)
    else:
        print('There are %s cases to train: '%number_to_quiz)

    
    for disease_folder_name in filtered_diseases: # for all diseases in the current directory           
       
        # if the disease matches the search
        if fnmatch.fnmatch(disease_folder_name, relevant_filenames) and disease_folder_name.startswith('[') and disease_folder_name.endswith(']'):    
            print ("""--------D-----------------------
                    \n-------------O------------------
                    \n------------------J-------------
                    \n-----------------------O--------""")
            disease = Disease(disease_folder_name)            
            disease.show_all_images()
            disease.ask_for_answers()
            #set_trace()   #for debugging
            key_press = {'i':disease.immunohistochemistry,
                         'd':disease.display_differentials,
                         'c':disease.differentials_challenge,
                        }    
            
            count += 1            

            print('%s cases completed. Quiz length: %s'%(count,number_to_quiz))
            if count == number_to_quiz:
                return

            looking_at_case = True
            
            while looking_at_case:
                post_answer_options = str(input('Press: Enter for next, d for ddx, c for ddx challenge, i for ihc, q to quit:'))
                

                #****************problem is here****************#
                #****************problem is here****************#
                #****************problem is here****************#
                #****************problem is here****************#
                if post_answer_options == 'q':  
                    looking_at_case = False
                    return
                elif post_answer_options == '': # enter gives ''
                    looking_at_case = False
                    break
                else:
                    try:
                        key_press[post_answer_options]() # the () executes the functions
                    except:
                        pass
            

def design_quiz(first_quiz=False):

    # Take inventory of the categories available (only do this once)
    if first_quiz == True:
        organ_list, disease_type_list, subtype_list, complexity_list, incidence_list  = get_inventory()
    

    # number to quiz  
    try:
        number_to_quiz = int(input("\n(To skip, press enter)\nMaximum number of diseases in the quiz: "))
    except ValueError:
        print('(Defaulting to 10 elements)\n')
        number_to_quiz = 10
    
    #[organ] skin, lung, brain
    print('Organs/systems available:')
    for option in organ_list:
        print('\n\t'+option)
    organ = str(input('''Please specify what you are interested in:
                        \n(To skip, press enter)
                        \nOrgan/system '''))
    while organ not in organ_list and organ != '':
            organ = str(input("\n(To skip, press enter)\n*Typo* Organ/sysem: "))
    
    #[type] inflammatory, benign_tumour, premalignant, malignant, congenital
    print('Disease types available:')
    for option in disease_type_list:
        print('\n\t'+option)
    disease_type = str(input("\n(To skip, press enter)\nDisease type: "))
    while disease_type not in disease_type_list and disease_type != '':
            disease_type = str(input("\n(To skip, press enter)\n*Typo* Disease type: "))
    
    #[subtype]     
    print('Subtypes available:')
    for option in subtype_list:
        print('\n\t'+option)
    subtype = str(input("\n(To skip, press enter)\nSubtype: "))
    while subtype not in subtype_list and subtype != '':
            subtype = str(input("\n(To skip, press enter)\n*Typo* Subtype: "))
    
    #[complexity] basic, advanced, spot_diagnosis, curiosity
    print('Complexities available:')
    for option in complexity_list:
        print('\n\t'+option)
    complexity = str(input("\n(To skip, press enter)\nComplexity: "))
    while complexity not in complexity_list and complexity != '':
            complexity = str(input("\n(To skip, press enter)\n*Typo* Complexity: "))
    
    #[incidence] common, uncommon (moderate), rare
    print('incidences available: ', incidence_list)
    for option in incidence_list:
        print('\n\t'+option)    
    incidence = str(input("\n(To skip, press enter)\nIncidence: "))
    while incidence not in incidence_list and incidence != '':
            incidence = str(input("\n(To skip, press enter)\n*Typo* incidence: "))
    
        
    results = {'organ': organ,
              'disease_type': disease_type,
              'subtype': subtype,
              'complexity': complexity,
              'incidence': incidence,
              'number_to_quiz': number_to_quiz}
    designed_quiz_preparer(**results)
    


In [ ]:
design_quiz(first_quiz=True)

In [ ]:
# Export as python .py file
# In shell use pyinstaller, run: pyinstaller --onefile file_name.py


Below here is for testing only

In [ ]:
print(os.getcwd())
# https://learnxinyminutes.com/docs/toml/
# test

folder = '[eye][benign][degenerative][spot_diagnosis][common][pingueculum]'
file = 'pingueculum.txt'

try:
    text_file_contents = {} # start with blank dictionary
    print('trying to open: '+folder+'/'+file)
    text_file_contents = toml.load(folder+'/'+file) # open file as read text only
except FileNotFoundError:
    print('Error: could not load the .txt file')
#except:
#    print('trouble reading some of the file')

try:
    description = text_file_contents['description']
    print(description)
except KeyError:
    print('(no description available)')
    
try:
    ddx_list = text_file_contents['differentials']
    for ddx_name in ddx_list:
        print(ddx_name)
except KeyError:
    print('(no differentials available)')

try:
    ihc_dictionary = text_file_contents['immunohistochemistry']
    for ihc_name in ihc_dictionary:
        print('The %s is %s'%(ihc_name, ihc_dictionary[ihc_name]))
except KeyError:
    print('(no ihc available)')




In [ ]:
# Goal: To call a function when 'g' is the user input

def go():
    hidden = "go time"
    return hidden

button = {'g':go} 


input_string = str(input('ENTER HERE:  '))

try:
    secret = button[input_string]()
except:
    pass

print('secret was:',secret)